In [2]:
import copy
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt

from DataPreprocess import create_dataloader
from InitializingModule import InitModel
from TrainingAlgorithm import train, LTS_module, multiclass_regularization, multiclass_weight_tuning, reorganize_module

In [3]:
df = pd.read_csv('data/SPECT_data.txt', header=None)

In [6]:
df

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,1,1,0,0,1,1,0,0,0,1,...,0,1,1,1,0,0,1,1,0,0
1,1,1,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,0,1,0,0,1,...,0,1,1,0,0,0,0,0,0,1
3,1,0,1,1,1,0,0,1,0,1,...,1,1,0,1,0,0,0,0,1,0
4,1,0,0,1,0,0,0,0,1,0,...,1,1,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
263,0,1,0,0,0,1,1,0,0,1,...,0,1,0,0,0,0,1,1,0,0
264,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
265,0,0,0,1,1,0,0,1,0,0,...,1,1,0,0,0,0,0,0,1,1


In [7]:
df.iloc[:, 0].value_counts()

1    212
0     55
Name: 0, dtype: int64

In [ ]:
def learning_mechanism(datapath, hidden_size, criterion, n):
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_loader, val_loader = create_dataloader(datapath)
    
    N_data = train_loader.dataset.X.shape[0]
    input_size = train_loader.dataset.X.shape[1]
    
    # initialize model
    init_model = InitModel(input_size, hidden_size, 1, device)
    model = init_model.init_module_multi_ReLU_AE(train_loader)
    
    while N_data > n_data:
        n_train_loader, n_data = LTS_module(train_loader=train_loader, model=model, criterion=criterion, n=n, device=device)
        
        if N_data <= n_data:
            return model
        
        param_num = sum(p.numel() for p in model.parameters())
        
        if n_data < param_num:
            # Add reorganizing model (longer)
            model = reorganize_module(model, n_train_loader, val_loader, criterion, reg_params, weight_params)
            continue
        
        saved_model = copy.deepcopy(model)
        situation, model = multiclass_weight_tuning(n_train_loader, val_loader, epochs, model, criterion, loss_threshold, eta_threshold)
        # model, situation = weight-tune
        
        if situation == 'Acceptable':
            # Add reorganizing model (longer)
            model = reorganize_module(model, n_train_loader, val_loader, criterion, reg_params, weight_params)
            continue
        
        model = saved_model
        
        # cramming
        
        # Add reorganizing model (shorter)
        model = reorganize_module(model, n_train_loader, val_loader, criterion, reg_params, weight_params, k=1, p=25)